In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Tue Oct 14 12:56:40 2025

@author: manny

Canada Economic Metrics Analyzer for Spyder
Three Timeframes: Long, Medium, Short Term
STRICTLY CANADA DATA
"""

import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import warnings
import sys
warnings.filterwarnings('ignore')

# Set up basic plotting style
plt.style.use('default')

class CanadaEconomicAnalyzer:
    def __init__(self):
        self.metrics_data = {}
        self.timeframe_options = {
            '2_years': '2 Years',
            '3_years': '3 Years', 
            '5_years': '5 Years',
            '10_years': '10 Years',
            '15_years': '15 Years', 
            '20_years': '20 Years',
            '25_years': '25 Years'
        }
        
        # Only the 4 requested metrics
        self.metrics = {
            'Unemployment Rate': 'LRUN64TTCAQ156S',  # Canada Unemployment Rate
            'CPI Inflation': 'CPALTT01CAQ659N',      # Canada Consumer Price Index
            'GDP Growth': 'NGDPRSAXDCCAQ',           # Canada Real GDP
            'Interest Rate': 'IRSTCI01CAQ156N'       # Canada Interest Rate
        }
    
    def get_date_range(self, timeframe):
        """
        Calculate date range based on selected timeframe
        """
        end_date = datetime.now()
        
        timeframe_map = {
            '2_years': 2,
            '3_years': 3,
            '5_years': 5,
            '10_years': 10,
            '15_years': 15,
            '20_years': 20,
            '25_years': 25
        }
        
        if timeframe in timeframe_map:
            start_date = end_date - timedelta(days=timeframe_map[timeframe]*365)
        else:
            start_date = end_date - timedelta(days=2*365)  # default to 2 years
        
        return start_date.strftime('%Y-%m-%d'), end_date.strftime('%Y-%m-%d')
    
    def fetch_economic_data(self, timeframe):
        """
        Fetch economic data for Canada for the specified timeframe
        """
        try:
            import pandas_datareader.data as web
        except ImportError:
            print("Installing pandas-datareader...")
            import subprocess
            subprocess.check_call([sys.executable, "-m", "pip", "install", "pandas-datareader"])
            import pandas_datareader.data as web
        
        start_date, end_date = self.get_date_range(timeframe)
        
        print(f"Fetching CANADA data for {self.timeframe_options[timeframe]} timeframe...")
        print(f"Date range: {start_date} to {end_date}")
        print("=" * 60)
        
        successful_metrics = 0
        failed_metrics = []
        
        for metric, code in self.metrics.items():
            try:
                data = web.DataReader(code, 'fred', start_date, end_date)
                if not data.empty:
                    self.metrics_data[metric] = data
                    print(f"✓ {metric} - Data points: {len(data)}")
                    successful_metrics += 1
                else:
                    print(f"✗ {metric} - No data available")
                    failed_metrics.append(metric)
            except Exception as e:
                # Suppress the verbose HTML error messages
                error_msg = str(e)
                if "Response Text" in error_msg:
                    # Extract just the main error part before the HTML
                    error_msg = error_msg.split("Response Text:")[0].strip()
                print(f"✗ {metric} - Error: {error_msg}")
                failed_metrics.append(metric)
        
        # Show which metrics failed
        if failed_metrics:
            print(f"\nFailed to fetch: {', '.join(failed_metrics)}")
        
        if successful_metrics == 0:
            print("\nNo real data could be fetched. Generating realistic sample data...")
            self.generate_sample_data(timeframe)
            successful_metrics = len(self.metrics_data)
        
        print(f"\nSuccessfully loaded {successful_metrics} out of {len(self.metrics)} metrics")
        return successful_metrics > 0
    
    def generate_sample_data(self, timeframe):
        """
        Generate realistic sample Canada economic data when real data isn't available
        """
        import numpy as np
        
        end_date = datetime.now()
        timeframe_map = {
            '2_years': 2,
            '3_years': 3,
            '5_years': 5,
            '10_years': 10,
            '15_years': 15,
            '20_years': 20,
            '25_years': 25
        }
        
        if timeframe in timeframe_map:
            start_date = end_date - timedelta(days=timeframe_map[timeframe]*365)
        else:
            start_date = end_date - timedelta(days=2*365)
        
        freq = 'M'
        dates = pd.date_range(start=start_date, end=end_date, freq=freq)
        np.random.seed(42)  # For consistent results
        
        # Generate realistic Canada economic data
        # Unemployment Rate Canada (5-12%)
        unemployment = 6.5 + np.random.normal(0, 1.0, len(dates)) + np.sin(np.arange(len(dates)) * 0.3) * 2
        self.metrics_data['Unemployment Rate'] = pd.DataFrame(
            {'Value': np.clip(unemployment, 5.0, 12.0)}, 
            index=dates
        )
        
        # CPI Inflation (1-8%)
        cpi = 2.5 + np.random.normal(0, 1.5, len(dates)) + np.cos(np.arange(len(dates)) * 0.25) * 3
        self.metrics_data['CPI Inflation'] = pd.DataFrame(
            {'Value': np.clip(cpi, 1.0, 8.0)}, 
            index=dates
        )
        
        # GDP Growth (-10% to +10%)
        gdp = 2.0 + np.random.normal(0, 3.0, len(dates)) + np.sin(np.arange(len(dates)) * 0.4) * 5
        self.metrics_data['GDP Growth'] = pd.DataFrame(
            {'Value': np.clip(gdp, -10.0, 10.0)}, 
            index=dates
        )
        
        # Interest Rate (0-10%)
        interest = 3.0 + np.random.normal(0, 1.5, len(dates)) + np.cos(np.arange(len(dates)) * 0.3) * 2
        self.metrics_data['Interest Rate'] = pd.DataFrame(
            {'Value': np.clip(interest, 0.0, 10.0)}, 
            index=dates
        )
    
    def plot_individual_metrics(self, selected_metrics=None):
        """
        Plot individual economic metrics
        """
        if not self.metrics_data:
            print("No data available. Please fetch data first.")
            return
        
        if selected_metrics is None:
            selected_metrics = list(self.metrics_data.keys())
        
        # Filter only available metrics
        selected_metrics = [m for m in selected_metrics if m in self.metrics_data]
        
        n_metrics = len(selected_metrics)
        if n_metrics == 0:
            print("No valid metrics selected.")
            return
        
        fig, axes = plt.subplots(n_metrics, 1, figsize=(14, 4*n_metrics))
        if n_metrics == 1:
            axes = [axes]
        
        for i, metric in enumerate(selected_metrics):
            data = self.metrics_data[metric]
            ax = axes[i]
            
            # Plot the data
            ax.plot(data.index, data.iloc[:, 0], linewidth=2.5, marker='o', markersize=3)
            ax.set_title(f'Canada - {metric}', fontsize=14, fontweight='bold', pad=20)
            ax.set_ylabel(metric, fontsize=12)
            ax.grid(True, alpha=0.3, linestyle='--')
            ax.tick_params(axis='x', rotation=45)
            
            # Add value annotation for the latest point
            last_date = data.index[-1]
            last_value = data.iloc[-1, 0]
            ax.annotate(f'{last_value:.2f}', 
                       xy=(last_date, last_value),
                       xytext=(15, 5), textcoords='offset points',
                       fontsize=11, fontweight='bold',
                       bbox=dict(boxstyle='round,pad=0.4', facecolor='lightblue', alpha=0.8))
        
        plt.tight_layout()
        plt.show()
    
    def plot_comprehensive_dashboard(self):
        """
        Create a comprehensive dashboard of all Canada economic metrics
        """
        if not self.metrics_data:
            print("No data available. Please fetch data first.")
            return
        
        metrics_list = list(self.metrics_data.keys())
        n_metrics = len(metrics_list)
        
        # Create subplot grid (adjust based on number of metrics)
        n_cols = 2
        n_rows = (n_metrics + n_cols - 1) // n_cols
        
        fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 5*n_rows))
        axes = axes.flatten()
        
        for i, metric in enumerate(metrics_list):
            if i < len(axes):
                data = self.metrics_data[metric]
                axes[i].plot(data.index, data.iloc[:, 0], linewidth=2, color=f'C{i}')
                axes[i].set_title(metric, fontsize=12, fontweight='bold')
                axes[i].grid(True, alpha=0.3)
                axes[i].tick_params(axis='x', rotation=45)
                
                # Add current value to title
                current_val = data.iloc[-1, 0]
                axes[i].set_title(f'{metric}\n(Current: {current_val:.2f})', fontsize=11, fontweight='bold')
        
        # Hide empty subplots
        for i in range(len(metrics_list), len(axes)):
            axes[i].set_visible(False)
        
        plt.suptitle('Canada Economic Metrics Dashboard', fontsize=16, fontweight='bold')
        plt.tight_layout()
        plt.show()
    
    def display_summary_statistics(self):
        """
        Display summary statistics for all metrics
        """
        if not self.metrics_data:
            print("No data available. Please fetch data first.")
            return
        
        print("\n" + "="*80)
        print("CANADA ECONOMIC METRICS - SUMMARY STATISTICS")
        print("="*80)
        
        for metric, data in self.metrics_data.items():
            if not data.empty:
                values = data.iloc[:, 0]
                print(f"\n{metric}:")
                print(f"  Latest Value: {values.iloc[-1]:.2f}")
                print(f"  Period: {data.index[0].strftime('%Y-%m-%d')} to {data.index[-1].strftime('%Y-%m-%d')}")
                print(f"  Average: {values.mean():.2f}")
                print(f"  Minimum: {values.min():.2f}")
                print(f"  Maximum: {values.max():.2f}")
                print(f"  Data Points: {len(values)}")

def main():
    """
    Main interactive function for Canada Economic Analysis
    """
    analyzer = CanadaEconomicAnalyzer()
    
    print("CANADA ECONOMIC METRICS ANALYZER")
    print("="*50)
    print("Timeframe Options:")
    print("1. 2 Years")
    print("2. 3 Years") 
    print("3. 5 Years")
    print("4. 10 Years")
    print("5. 15 Years")
    print("6. 20 Years")
    print("7. 25 Years")
    print("="*50)
    
    while True:
        try:
            choice = input("\nSelect timeframe (1-7) or 'q' to quit: ").strip()
            
            if choice.lower() == 'q':
                print("Goodbye!")
                return  # Use return instead of break to exit completely
            
            timeframe_map = {
                '1': '2_years', 
                '2': '3_years', 
                '3': '5_years',
                '4': '10_years',
                '5': '15_years',
                '6': '20_years', 
                '7': '25_years'
            }
            
            if choice in timeframe_map:
                timeframe = timeframe_map[choice]
                
                # Fetch data
                success = analyzer.fetch_economic_data(timeframe)
                
                if success:
                    # Analysis menu with proper exit handling
                    analysis_complete = False
                    while not analysis_complete:
                        print("\nAnalysis Options:")
                        print("1. Plot Individual Metrics")
                        print("2. Comprehensive Dashboard")
                        print("3. Summary Statistics")
                        print("4. Change Timeframe")
                        print("5. Exit Program")
                        
                        analysis_choice = input("\nSelect option (1-5): ").strip()
                        
                        if analysis_choice == '1':
                            print(f"\nAvailable metrics: {', '.join(analyzer.metrics.keys())}")
                            metric_input = input("Enter metrics to plot (comma-separated) or 'all' for all: ")
                            if metric_input.lower() == 'all':
                                analyzer.plot_individual_metrics()
                            else:
                                selected = [m.strip() for m in metric_input.split(',')]
                                analyzer.plot_individual_metrics(selected)
                        
                        elif analysis_choice == '2':
                            analyzer.plot_comprehensive_dashboard()
                        
                        elif analysis_choice == '3':
                            analyzer.display_summary_statistics()
                        
                        elif analysis_choice == '4':
                            analysis_complete = True  # Exit analysis loop to change timeframe
                            print("Returning to timeframe selection...")
                        
                        elif analysis_choice == '5':
                            print("Goodbye!")
                            return  # Exit program completely
                        
                        else:
                            print("Invalid choice. Please select 1-5.")
                
                else:
                    print("Failed to fetch sufficient data. Please try a different timeframe.")
            
            else:
                print("Invalid choice. Please select 1-7, or 'q' to quit.")
                
        except KeyboardInterrupt:
            print("\n\nProgram interrupted by user. Goodbye!")
            return
        except Exception as e:
            print(f"An unexpected error occurred: {e}")
            print("Returning to main menu...")
            continue  # Continue to next iteration instead of breaking

# Run the analysis
if __name__ == "__main__":
    main()


CANADA ECONOMIC METRICS ANALYZER
Timeframe Options:
1. 2 Years
2. 3 Years
3. 5 Years
4. 10 Years
5. 15 Years
6. 20 Years
7. 25 Years
